In [1]:
import pandas as pd

In [2]:
df_fusionné = pd.read_csv('df_fusionné.csv')

In [3]:
df_fusionné = df_fusionné.sample(frac=1)


In [4]:
df_anormal = df_fusionné[df_fusionné["ColumnName"] != "N"]

In [5]:
value_counts = df_anormal["ColumnName"].value_counts()
print(value_counts)

ColumnName
Q    16086
V    14472
S     5562
F     1606
Name: count, dtype: int64


In [6]:
# Assuming df_anormal is your DataFrame and 'ColumnName' is the column with class labels
# The target size is the smallest class count, which is 1606 for class 'F'
target_size = 1606

# Sample each class in the DataFrame to have the same number of samples as the smallest class
balanced_df = pd.DataFrame()  # This will be the new DataFrame with balanced classes

for label in df_anormal['ColumnName'].unique():
    df_subset = df_anormal[df_anormal['ColumnName'] == label]
    balanced_subset = df_subset.sample(n=target_size, random_state=42)  # Use a fixed seed for reproducibility
    balanced_df = pd.concat([balanced_df, balanced_subset])

# Reset index of the new DataFrame
balanced_df.reset_index(drop=True, inplace=True)

print("New class counts:")
print(balanced_df['ColumnName'].value_counts())


New class counts:
ColumnName
S    1606
V    1606
Q    1606
F    1606
Name: count, dtype: int64


In [7]:
balanced_df

,ColumnName,0,1,2,3,4,5,6,7,8,...,1430,1431,1432,1433,1434,1435,1436,1437,1438,1439
0,S,0.498203,0.499715,0.501397,0.503192,0.504812,0.506426,0.508057,0.509718,0.511629,...,0.491440,0.492323,0.492928,0.493435,0.494023,0.494621,0.495158,0.495754,0.496299,0.496956
1,S,0.509112,0.509554,0.509776,0.509790,0.509856,0.509863,0.509730,0.509524,0.509289,...,0.490387,0.493197,0.495310,0.497184,0.498866,0.500450,0.502430,0.504401,0.506403,0.508225
2,S,0.477176,0.477812,0.478485,0.479221,0.480026,0.480862,0.481766,0.482679,0.483423,...,0.475006,0.475204,0.475368,0.475521,0.475720,0.475935,0.476093,0.476274,0.476447,0.476677
3,S,0.519417,0.523367,0.528685,0.533486,0.531742,0.527716,0.522264,0.516466,0.515130,...,0.503931,0.503267,0.504432,0.506327,0.508881,0.511642,0.513296,0.514818,0.515341,0.515902
4,S,0.508610,0.508441,0.508229,0.507993,0.507802,0.507606,0.507409,0.507194,0.506871,...,0.508809,0.508763,0.508760,0.508771,0.508771,0.508767,0.508770,0.508761,0.508759,0.508734
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6419,F,0.470792,0.474672,0.464490,0.476011,0.471428,0.466360,0.470277,0.476566,0.475796,...,0.585012,0.580923,0.585440,0.592865,0.592314,0.587616,0.595581,0.597872,0.585737,0.585435
6420,F,0.472230,0.483557,0.478036,0.467935,0.467892,0.470304,0.470565,0.470559,0.470530,...,0.456231,0.455535,0.456373,0.457901,0.460120,0.462486,0.464579,0.467114,0.466050,0.463491
6421,F,0.519919,0.517292,0.514850,0.512352,0.509429,0.506582,0.503527,0.500776,0.499948,...,0.535902,0.533823,0.532150,0.530639,0.528992,0.527366,0.526015,0.524669,0.523452,0.522144
6422,F,0.489571,0.491727,0.494136,0.496677,0.499145,0.501713,0.504294,0.507008,0.510383,...,0.481971,0.479574,0.478877,0.479059,0.479683,0.480777,0.482036,0.483541,0.485404,0.487500


In [16]:
X = df_anormal.iloc[:, 1:]
y = df_anormal.iloc[:, 0]

In [19]:
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout


label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

# Train
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [24]:
y_test

array([[0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       ...,
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.]], dtype=float32)

In [25]:
X_train.shape[1]

2160

In [26]:
y_train.shape[1]

4

In [27]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam

In [37]:
model = Sequential([
    Conv1D(filters=64, kernel_size=6, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=3),
    BatchNormalization(),

    Conv1D(filters=128, kernel_size=6, activation='relu'),
    MaxPooling1D(pool_size=2),
    BatchNormalization(),

    Conv1D(filters=128, kernel_size=6, activation='relu'),
    MaxPooling1D(pool_size=2),
    BatchNormalization(),

    Conv1D(filters=64, kernel_size=6, activation='relu'),
    MaxPooling1D(pool_size=2),
    BatchNormalization(),

    Flatten(),

    Dense(64, activation='relu'),

    Dense(32, activation='relu'),

    Dense(4, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['f1'])


# Summary of the model
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_15 (Conv1D)          (None, 2155, 64)          448       
                                                                 
 max_pooling1d_15 (MaxPooli  (None, 718, 64)           0         
 ng1D)                                                           
                                                                 
 batch_normalization_15 (Ba  (None, 718, 64)           256       
 tchNormalization)                                               
                                                                 
 conv1d_16 (Conv1D)          (None, 713, 128)          49280     
                                                                 
 max_pooling1d_16 (MaxPooli  (None, 356, 128)          0         
 ng1D)                                                           
                                                      

In [38]:
from tensorflow.keras.callbacks import EarlyStopping

# early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Example for the medium complexity model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc}')

Epoch 1/10
354/354 [==============================] - 14s 36ms/step - loss: 0.4498 - accuracy: 0.9100 - val_loss: 0.7126 - val_accuracy: 0.8031
Epoch 2/10
354/354 [==============================] - 12s 34ms/step - loss: 0.3175 - accuracy: 0.9443 - val_loss: 0.2777 - val_accuracy: 0.9465
Epoch 3/10
354/354 [==============================] - 12s 34ms/step - loss: 1.0670 - accuracy: 0.9250 - val_loss: 1.4973 - val_accuracy: 0.9335
Epoch 4/10
354/354 [==============================] - 12s 35ms/step - loss: 5.7875 - accuracy: 0.8975 - val_loss: 6.7188 - val_accuracy: 0.9316
Epoch 5/10
354/354 [==============================] - 12s 35ms/step - loss: 31.8966 - accuracy: 0.8734 - val_loss: 16.1506 - val_accuracy: 0.8741
Epoch 6/10
354/354 [==============================] - 12s 35ms/step - loss: 308.8495 - accuracy: 0.7129 - val_loss: 29.9282 - val_accuracy: 0.8301
Epoch 7/10
354/354 [==============================] - 12s 35ms/step - loss: 38.4793 - accuracy: 0.7992 - val_loss: 28.1875 - val_ac